The Following reads the FASTA file, and displays information on it

In [2]:
import os
from Bio import SeqIO

# Folder containing the unzipped FASTA files
folder_path = "./unzipped_mt_datasets/"

# List to store genome IDs with full species names
genomes = []

# Extract full header information for clarity
for filename in os.listdir(folder_path):
    if filename.endswith(".fa") or filename.endswith(".fasta"):
        file_path = os.path.join(folder_path, filename)
        record = next(SeqIO.parse(file_path, "fasta"))
        # Store the file name and actual sequence
        genomes.append((filename, str(record.seq)))  # Include the sequence as a string


# Print the loaded genomes to verify
print(f"Loaded genomes:")
for genome_id in genomes:
    print(genome_id[0])


Loaded genomes:
Danio_rerio.GRCz11.dna.chromosome.MT.fa
Gallus_gallus_gca016700215v2.bGalGal1.pat.whiteleghornlayer.GRCg7w.dna.nonchromosomal.fa
Homo_sapiens.GRCh38.dna.chromosome.MT.fa
Leptobrachium_leishanense.ASM966780v1.dna.nonchromosomal.fa
Mus_musculus.GRCm39.dna.chromosome.MT.fa
Sus_scrofa.Sscrofa11.1.dna.nonchromosomal.fa


splits the sequence into 6-mers and displays a sample

In [3]:
from collections import Counter

def k_mer_count(sequence, k=6):
    """Counts k-mers in the sequence."""
    return Counter([sequence[i:i + k] for i in range(len(sequence) - k + 1)])

# Generate k-mer counts for each genome
kmer_vectors = []
k = 6  # Define k value
for genome_id, sequence in genomes:
    if len(sequence) >= k:  # Only process sequences long enough for k-mers
        kmer_counts = k_mer_count(sequence, k=k)
        kmer_vectors.append(kmer_counts)
    else:
        kmer_vectors.append(Counter())  # Add an empty Counter for short sequences

# Print a single example at the end
if kmer_vectors:
    first_non_empty = next((kv for kv in kmer_vectors if kv), None)  # Find the first non-empty Counter
    if first_non_empty:
        print(f"Sample 6-mer counts: {list(first_non_empty.items())[:10]}")
    else:
        print("No valid k-mers found in the data.")
else:
    print("No data to process.")


Sample 6-mer counts: [('ACGGCC', 3), ('CGGCCG', 3), ('GGCCGG', 1), ('GCCGGC', 3), ('CCGGCG', 1), ('CGGCGA', 1), ('GGCGAC', 2), ('GCGACA', 2), ('CGACAA', 4), ('GACAAT', 8)]


transforming the k-mer count into a feature vector for clustering

In [4]:
from sklearn.feature_extraction import DictVectorizer

# Convert the k-mer dictionaries to numerical vectors
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(kmer_vectors)

print(f"Feature Vector Shape: {X.shape}")



Feature Vector Shape: (6, 4102)


In [17]:
from sklearn.cluster import KMeans

# Apply K-means clustering with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42).fit(X)

# Output the cluster labels for each genome
for i, (genome_id, _) in enumerate(genomes):
    print(f"{genome_id} -> Cluster {kmeans.labels_[i]}")



Danio_rerio.GRCz11.dna.chromosome.MT.fa -> Cluster 0
Gallus_gallus_gca016700215v2.bGalGal1.pat.whiteleghornlayer.GRCg7w.dna.nonchromosomal.fa -> Cluster 0
Homo_sapiens.GRCh38.dna.chromosome.MT.fa -> Cluster 0
Leptobrachium_leishanense.ASM966780v1.dna.nonchromosomal.fa -> Cluster 2
Mus_musculus.GRCm39.dna.chromosome.MT.fa -> Cluster 0
Sus_scrofa.Sscrofa11.1.dna.nonchromosomal.fa -> Cluster 1


In [18]:
from sklearn.metrics import silhouette_score

# Assuming you already have:
# - X: The feature matrix you used in K-means (your k-mer vectors or genomic features)
# - kmeans: Your trained K-means model

# Compute the Silhouette Score
sil_score = silhouette_score(X, kmeans.labels_)

# Display the score
print(f"Silhouette Score: {sil_score}")

# Provide an interpretation based on the score
if sil_score > 0.5:
    print("Good clustering! Your clusters are well-separated.")
elif 0.2 <= sil_score <= 0.5:
    print("Moderate clustering. There may be some overlap between clusters.")
else:
    print("Poor clustering. Consider adjusting the number of clusters or using a different algorithm.")


Silhouette Score: 0.49145614271990024
Moderate clustering. There may be some overlap between clusters.
